# OpenAI Function Calling In LangChain

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic 데이터 클래스는 파이썬의 데이터 클래스와 Pydantic의 validation 능력을 합쳐놓은 것입니다.

이는 명시된 타입이나 제약이 지켜질 수 있도록 보증함과 동시에 데이터 구조를 정의하는 간결한 방식을 제시합니다.

표준 파이썬에서 여러분은 아래와 같은 방식으로 클래스를 생성합니다:

In [ ]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [ ]:
foo.name

`__init__` 메서드에 `age`가 `int` 자료형이어야 함을 명시해두었음에도 불구하고,

`str` 자료형의 값을 전달해도 에러가 발생하지 않습니다.

In [ ]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [ ]:
foo.age

`pydantic`의 `BaseModel`을 상속받습니다.

In [ ]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [ ]:
foo_p.name

`age`에 `int`를 전달하지 않아 오류가 발생하여 인스턴스를 생성하지 못합니다.

In [ ]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

In [ ]:
class Class(BaseModel):
    students: List[pUser]

In [ ]:
# argument에 리스트를 전달할 수도 있습니다.
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [ ]:
obj

## Pydantic to OpenAI function definition
Pydantic으로 간단하게 선언한 함수를 OpenAI에서 필요로 하는 json 자료형으로 변환하는 과정에 대해 배웁니다.

In [ ]:
class WeatherSearch(BaseModel):
    # docstring은 description으로 변합니다.
    """Call this with an airport code to get the weather at that airport"""
    # Field description은 properties의 description으로 변합니다.
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_function

In [ ]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

Note: 다음 셀은 실행하면 에러가 발생합니다.

이는 `WeatherSearch1` 클래스에 `description`이 없기 때문입니다.

In [ ]:
convert_pydantic_to_openai_function(WeatherSearch1)

In [ ]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

위에서와 달리 랭체인에서 aurgment의 `description`은 optional이기 때문에 에러가 발생하지 않습니다.

In [ ]:
convert_pydantic_to_openai_function(WeatherSearch2)

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI()

model에 function을 전달하기 위한 두 가지 방법이 있습니다.

In [ ]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

In [ ]:
model_with_function = model.bind(functions=[weather_function])

In [ ]:
model_with_function.invoke("what is the weather in sf?")

## function 사용 강제하기

모델이 function을 반드시 사용하도록 강제할 수도 있습니다.

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
# 'hi'는 weather_function과 관계가 없지만 해당 함수를 호출한 결과가 나타나는 것을 확인할 수 있습니다.
model_with_forced_function.invoke("hi!")

## Using in a chain

우리는 이 함수에 묶인 모델을 chain으로 엮어 사용할 수 있습니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
# 여기서 `model_with_function`은 `model.bind`를 통해 `weather_function`을 포함하고 있습니다.
chain = prompt | model_with_function

In [ ]:
chain.invoke({"input": "what is the weather in sf?"})

## Using multiple functions

심지어 여러 개의 function을 묶어서 전달할 수 있습니다.

LLM은 question context에 근거하여 어떤 함수를 사용할지 결정하게 됩니다.

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [ ]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [ ]:
model_with_functions = model.bind(functions=functions)

각 question에 필요한 함수가 호출되는 결과를 확인할 수 있습니다.

In [ ]:
model_with_functions.invoke("what is the weather in sf?")

In [ ]:
model_with_functions.invoke("what are three songs by taylor swift?")

호출될 함수가 없다면 LLM이 직접 답변을 생성하게 됩니다.

In [ ]:
model_with_functions.invoke("hi!")